In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from workshop_helpers.utils import get_unique_name

credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

unique_name = get_unique_name(credential)


Found the config file in: /config.json


Validicating credential....
Credential Validated
Determining unique name....
Unique name: jmerkow


In [3]:
from azure.ai.ml.entities import AmlCompute

gpu_cluster_name = f"gpucluster-{unique_name}" ## TODO Add unique name
try:
    compute_target = ml_client.compute.get(gpu_cluster_name)
    print(f"Using existing GPU compute cluster: {gpu_cluster_name}")
except Exception as e:
    print(f"Creating new GPU compute cluster: {gpu_cluster_name}")
    compute_target = AmlCompute(
        name=gpu_cluster_name,
        size="Standard_NC6s_v3",  # GPU-enabled VM size, adjust if needed
        min_instances=0,
        max_instances=5,
    )
    ml_client.compute.begin_create_or_update(compute_target).result()
    print(f"Created GPU compute cluster: {gpu_cluster_name}")


Using existing GPU compute cluster: gpucluster-jmerkow


In [4]:
from azure.ai.ml.entities import Environment
from azure.ai.ml import command

# This assumes you have an 'environment.yaml' in your code folder that defines your Conda dependencies.
gpu_environment = Environment(
    name=f"uw-workshop-gpu-env-{unique_name}",
    description="GPU enabled environment",
    image="mcr.microsoft.com/azureml/curated/acpt-pytorch-1.13-cuda11.7:latest",
    conda_file="../03_Training/src/environment.yml",
    tags={"gpu": "true"}
)

# Register or update the environment in your workspace.
registered_env = ml_client.environments.create_or_update(gpu_environment)
print(f"Registered environment: {registered_env.name}:{registered_env.version}")

job = command(
    code=".",
    command="python -c \"print('hello')\"",
    environment=registered_env,
    compute=gpu_cluster_name,
    display_name=f"force_build_env-{unique_name}",
    experiment_name="ConnectivityTests",
)

submitted_job = ml_client.jobs.create_or_update(job)
submitted_job


Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Registered environment: uw-workshop-gpu-env-jmerkow:6


Uploading 01_EnvironmentSetup (0.02 MBs): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15114/15114 [00:00<00:00, 549315.97it/s]


Experiment,Name,Type,Status,Details Page
ConnectivityTests,clever_picture_tzg7x11y2d,command,Starting,Link to Azure Machine Learning studio


In [ ]:
submitted_job


Experiment,Name,Type,Status,Details Page
ConnectivityTests,gifted_cow_fwgzf4wfw3,command,Starting,Link to Azure Machine Learning studio


In [ ]:
import os
from azureml.fsspec import AzureMachineLearningFileSystem

# Get the data asset we registered
data_asset = ml_client.data.get(name="rsna-pneumonia-detection-challenge-zip", version="1")

# Get the data asset URI
data_uri = data_asset.path

# Create a directory to download the data
download_dir = "/home/azureuser/data-test"
os.makedirs(download_dir, exist_ok=True)

# Initialize the AzureMachineLearningFileSystem and download the file
fs = AzureMachineLearningFileSystem(data_uri)
destination_file = os.path.join(download_dir, "rsna-pneumonia-detection-challenge.zip")
if not os.path.exists(destination_file):
	fs.download(data_uri, download_dir)
else:
	print(f"File already exists at: {destination_file}")

print(f"Data downloaded to: {destination_file}")
print(destination_file)
if os.path.exists(destination_file):
    print(f"File size: {os.path.getsize(destination_file) / (1024 **3):.2f} GB")
